# Рекомендация тарифов

## Откройте и изучите файл

In [1]:
#необходимые библиотеки
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.dummy import DummyClassifier

In [2]:
#прочитаем файл 
try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(df.head(10))
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные состоят из 3214 строк и 5 столбцов. Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Целевой признак в построении модели - столбец **is_ultra** (качественный тип данных). Перед нами стоит задача классификации.
Пропусков в данных нет. Предобработка по условию не нужна, но это не отменяет дополнительную подготовку данных.

Еще раз обратим внимание на данные.

In [4]:
#убедимся, что нет явных дубликатов
df.duplicated().sum()

0

In [5]:
#посмотрим на значения
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Очевидно, что присутствуют нулевые значения в колонках с признаками объектов. Нули в сообщениях не беспокоят, поскольку в настоящее время смс-ки мало кто пишет, нули в данных мобильного интернет трафика заменили пропущенные значения видимо. Другое дело нулевые значения в 'calls' и 'minute'. 	

In [6]:
#посмотрим на нулевые значение столюцов 'calls' и 'minute', тем более,
#что вероятнее всего они будут совпадать
df[df['minutes']==0]

,calls,minutes,messages,mb_used,is_ultra
54,0.0,0.0,33.0,14010.33,1
247,0.0,0.0,35.0,16444.99,1
264,0.0,0.0,21.0,19559.55,0
351,0.0,0.0,8.0,35525.61,1
390,0.0,0.0,25.0,19088.67,1
484,0.0,0.0,191.0,32448.02,1
551,0.0,0.0,24.0,18701.54,1
647,0.0,0.0,30.0,10236.20,1
769,0.0,0.0,32.0,17206.44,0
884,0.0,0.0,180.0,32045.73,1


А вот данные о сообщениях и интернет трафике в строках с нулевыми значения 'calls' и 'minute' присутствуют, значит нулями заменили пропуски в 'calls' и 'minute'. Считаю, что такие данные могут негативно повлиять на результат обучения модели.

In [7]:
#удалим строки значений 'minute' == 0
df = df.query('minutes > 0')
#проверим
df[df['minutes']==0]['minutes'].count()

0

Сейчас необходимо оценить взаимосвязи между признаками, исключая целевой


In [8]:
#делаем копию датасета
df_features = df.copy()

#удаляем целевой признак
del df_features['is_ultra']

#сформируем матрицу корреляции признаков
round(df_features.corr(), 2).style.background_gradient('coolwarm')

,calls,minutes,messages,mb_used
calls,1.000000,0.980000,0.200000,0.300000
minutes,0.980000,1.000000,0.190000,0.300000
messages,0.200000,0.190000,1.000000,0.190000
mb_used,0.300000,0.300000,0.190000,1.000000


Сильная корреляция между признаками  'calls' и 'minute' - это логично. Такие пары переменных избыточны, а явление носит название мультиколлинеарности. В этой ситуации оценки коэффициентов (параметров) модели могут случайно и значительно изменяться даже при небольших изменениях в исходных данных, т.е. решение становится неустойчивым.
Учитывая это, колонку 'calls' необходимо удалить в исходном датасете, тем более, что количество звонков не является определяющим и информативным параметром тарифа, а остальные признаки являются.

In [9]:
#удаляем столбец
del df['calls']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3174 entries, 0 to 3213
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   minutes   3174 non-null   float64
 1   messages  3174 non-null   float64
 2   mb_used   3174 non-null   float64
 3   is_ultra  3174 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 124.0 KB


Вывод. Мы изучили файл с данными. Определили целевой признак в построении модели - это столбец 'is_ultra'. Проверили признаки на наличие явных дубликатов. Удалили строки с нулевыми значениями количества минут, чтобы улучшить обучаемость модели. Исключили мультиколлинеарность признаков путем удаления колонки 'call'. 
Данные готовы.

## Разбейте данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки, потому что спрятанной тестовой выборки нет. Исходные данные разбивают в соотношении 3:1:1.

Для этого предусмотрена функция train_test_split. Она разбивает любой датасет на обучающую и тестовую выборки.Нам нужно разделить датасет на три части и деление будет происходить в два этапа. Сначала разделим на обучающий набор и проверочный в соотношении 60% на 40%. Затем проверочный набор поделим пополам на валидационный набор и тестовый.

Зададим параметр random_state чтобы можно было воспроизвести это псевдослучайно разбиение данных.

In [10]:
#делим датасет
df_train, df_check = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_check, test_size=0.5, random_state=12345)

In [11]:
#проверим
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1904, 4)
(635, 4)
(635, 4)


Вывод. Мы получили 3 (три) датасета.
df_train  - для обучения модели
df_valid - для поверки модели на переобучение
df_test - для оценки качества модели

## Исследуйте модели

Как мы обозначили ранее целевым признаком является столбец 'is_ultra'. Этот признак является категориальным. Следовательно нам необходимо решить задачу классификации.

В процессе моделирования будем использовать следующие модели машинного обучения - DecisionTreeClassifier,  RandomForestClassifier, LogisticRegression.

Алгоритмы "Решающего дерева" и "Случайного леса" имеют гиперпараметры, меняя которые можно подобрать наилучшую модель. У "Решающего дерева" это максимальная глубина дерева max_depth. У "Случайного леса" к максимальной глубине добавляется еще один гиперпараметр количество деревьев n_estimators.

Каждую модель обучим на обучающем наборе и проверим на валидационной выборке и определим победителя. 

In [12]:
# объявим переменные features и target (признаки и целевой признак)
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Модель "Решающего дерева". Гиперпараметр max_depth будем изменять в пределах от 1 до 10 в цикле.

In [13]:
leader_model = 0
best_depth = 0
best_result = 0
for depth in (range(1, 11)):
    model = DecisionTreeClassifier(random_state=12345, \
                                       max_depth = depth).fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(predictions_valid, target_valid)
    if best_result < result:
        leader_model = model
        best_depth = depth
        best_result = result
print('best max_depth =', best_depth, ';','best Accuracy = ', best_result, end ='') 

best max_depth = 4 ; best Accuracy =  0.7779527559055118

Accuracy лучшей модели из десяти равно 0.7779527559055118, с гиперпараметром max_depth равным- 4.

Модель "Случайный лес". Первый цикл будет перебирать глубину дерева max_depth, а второй для каждой глубины дерева будет перебирать количества деревьев n_estimators. Гиперпараметры будем менять от 1 до 10.

In [14]:
leader_model = 0
best_est = 0
best_depth = 0
best_result = 0
for est in (range(1, 11)):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)\
        .fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(predictions_valid, target_valid)
        if best_result < result:
            leader_model = model
            best_est = est
            best_depth = depth
            best_result = result

print('best n_estimators =', best_est, ';', \
      'best max_depth =', best_depth,';',
      'best Accuracy = ', best_result, end ='') 

best n_estimators = 8 ; best max_depth = 6 ; best Accuracy =  0.7921259842519685

Accuracy лучшей модели из десяти равно 0.7921259842519685, с гиперпараметром max_depth равным-  6 и гиперпараметром n_estimators равным- 8.

Модель "Логистическая регрессия".

In [15]:
model_lr = LogisticRegression(random_state=12345).fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(predictions_valid_lr, target_valid)
print('Accuracy:', accuracy_lr, end ='')

Accuracy: 0.7401574803149606

Accuracy модели LogisticRegression равно 0.7401574803149606.

Вывод.
Лучший результат Accuracy у модели решающего дерева равен 0.7779527559055118 (второе место).
Лучший результат Accuracy у модели случайного леса равен 0.7921259842519685 (первое место).
Accuracy у модели логистической регрессии равен 0.7401574803149606 (худший результат).

## Проверьте модель на тестовой выборке

Победитель модель RandomForestClassifier с результатом 0.7921259842519685% правильных ответов, проверим ее на тестовой выборке.

In [16]:
model_rfc = RandomForestClassifier(random_state=12345, n_estimators = 8, max_depth = 6)\
.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result_test = accuracy_score(predictions_test, target_test)
print('Accuracy:', result_test)

Accuracy: 0.8031496062992126


Вывод. Accuracy у модели RandomForestClassifier на тестовой выборке равна 0.8078740157480315 (немногим больше чем на обучающем датасете). Это значит, что у тестовой выборки репрезентативность выше чем у обучающей. Оценку качества, сделанную по тестовой выборке можно применить для выбора лучшей модели.

## (бонус) Проверьте модели на адекватность

In [17]:
# для выбора модели посмотрим на категории в признаке'is ultra'
df['is_ultra'].value_counts()

0    2223
1     951
Name: is_ultra, dtype: int64

Видим, что клиентов на тарифе "Смарт" более чем в 2 (два) раза больше, чем на "Ультра". Поэтому ниже.

Ввиду дисбаланса данных для проверки попробуем использовать модель DummyClassifier. DummyClassifier - это классификатор в библиотеке sklearn, который делает прогнозы с использованием простых правил и не генерирует никаких ценных сведений о данных. Такие классификаторы используются в качестве базовой линии и могут быть сравнены с реальными классификаторами. Другими словами если Accuracy DummyClassifier ниже Accuracy нашей модели RandomForestClassifie, то наша модель адекватна (вменяема).

In [18]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dc)

Accuracy DummyClassifier: 0.7039370078740157


Accuracy DummyClassifier < Accuracy RandomForestClassifier. Наша модель адекватна. 

**Общий вывод**. 
В проекте мы дополнительно обработали данные. Выявили избыточные признаки. Несомненно это позволило модели надежнее выявлять закономерности в признаках и увеличило ее качество, не отвлекаясь на шумы.
Было проверено три модели: "Решающее дерево", "Случайный лес" и "Логистическая регрессия". В каждой модели, изменяя ее гиперпараметры, мы нашли нашли лучший вариант с наибольшим количеством правильных ответов. Сравнив лучшие модели по доле правильных ответов мы определили, что для нашей задачи подходит модель RandomForestClassifier, n_estimators = 8 , max_depth = 6, Accuracy =  0.7921259842519685.
Мы оценили качество лучшей модели RandomForestClassifier на тестовом наборе данных (результат составил 0.8031496062992126% правильных ответов) и сравнили его с результатом модели DummyClassifier (0.7039370078740157% правильных ответов), убедившись, что наша модель адекватна.
Использовать данную модель для рекомендации тарифов "Смарт" или "Ультра" разрешаю)).